In [152]:
from tulip import tlp
from sympy import *
from sympy.geometry import Segment, Point, Circle, Line, intersection

EDGE_OFFSET = 0.1

In [153]:
graph = tlp.loadGraph('test0.tlp')

In [154]:
print [x for x in graph.getNodes()]
print [x for x in graph.getDescendantGraphs()]

[<node 0>, <node 1>, <node 2>, <node 3>, <node 4>, <node 5>, <node 6>]
[<graph "top" (id 1) >, <graph "mid" (id 3) >, <graph "mid1" (id 2) >, <graph "bot" (id 4) >]


In [155]:
viewLayout = graph.getLayoutProperty('viewLayout')
viewSize = graph.getSizeProperty('viewSize')

In [156]:
def crossLineSegmentAndCircle(seg, cir):
    d = seg.distance(cir.center)
    if d < cir.radius:
        intersectLength = sqrt(cir.radius * cir.radius - d * d)
        return intersectLength * EDGE_OFFSET
    else: 
        return 0

In [157]:
def getNodeOverlap(e, v):
    s = viewLayout[graph.source(e)]
    t = viewLayout[graph.target(e)]
    seg = Segment(Point(s.x(), s.y()), Point(t.x(), t.y()))
    nodeCenter = Point(viewLayout[v].x(), viewLayout[v].y())
    radius = viewSize[v][0] / 2
    return crossLineSegmentAndCircle(seg, Circle(nodeCenter, radius))

In [158]:
n = graph.nodes()[1]
e = graph.edges()[5]

In [159]:
getNodeOverlap(e, n)

0.0486960260529321

In [160]:
def getSubGraphOverlap(e, g):
    # print 'Calculating overlap between edge {} and subgraph {}'.format(e, g)
    s = viewLayout[graph.source(e)]
    t = viewLayout[graph.target(e)]
    seg = Segment(Point(s.x(), s.y()), Point(t.x(), t.y()))
    bc = tlp.computeBoundingRadius(g)
    graphCenter = Point(bc[0].x(), bc[1].y())
    radius = bc[0].dist(bc[1]) / 2.0
    # print 'bounding circle: ', graphCenter, radius
    return crossLineSegmentAndCircle(seg, Circle(graphCenter, radius))

In [161]:
g = graph.getDescendantGraph('mid')
print [viewLayout[x] for x in g.getNodes()]
print getSubGraphOverlap(e, g)

[(-2.81762,6.34367,0), (-3.55984,3.58271,0), (0.46692,2.99473,0)]
0.0889189810731178


In [167]:
def getSubGraphPenalty(e, g):
    print '\t',e, g
    sum = getSubGraphOverlap(e, g)
    for s in g.getDescendantGraphs():
        sum += getSubGraphOverlap(e, s)
    for n in g.getNodes():
        sum += getNodeOverlap(e, n)
    return sum

In [168]:
getSubGraphPenalty(e, g)

	<edge 5> <graph "mid" (id 3) >


0.244443065194017

In [169]:
def getGraphPenalty(g):
    penalty = 0
    for e in graph.getEdges():
        s, t = graph.ends(e)
        p = 0
        
        for subGraph in graph.getDescendantGraphs():
            if not subGraph.isElement(s) and not subGraph.isElement(t):
                p += getSubGraphPenalty(e, subGraph)
        
        for node in graph.getNodes():
            if node != s and node != t:
                p += getNodeOverlap(e, node)
                
        print 'penalty for edge {} is {}'.format(e, p)
        penalty += p
    return penalty

In [170]:
print getGraphPenalty(graph)

	<edge 0> <graph "top" (id 1) >
	<edge 0> <graph "bot" (id 4) >
penalty for edge <edge 0> is 0
	<edge 1> <graph "top" (id 1) >
	<edge 1> <graph "bot" (id 4) >
penalty for edge <edge 1> is 0
	<edge 2> <graph "top" (id 1) >
	<edge 2> <graph "bot" (id 4) >
penalty for edge <edge 2> is 0
	<edge 3> <graph "mid" (id 3) >
	<edge 3> <graph "mid1" (id 2) >
	<edge 3> <graph "bot" (id 4) >
penalty for edge <edge 3> is 0
	<edge 4> <graph "top" (id 1) >
	<edge 4> <graph "mid" (id 3) >
	<edge 4> <graph "mid1" (id 2) >
penalty for edge <edge 4> is 0
	<edge 5> <graph "mid" (id 3) >
	<edge 5> <graph "mid1" (id 2) >
penalty for edge <edge 5> is 0.448663175367848
	<edge 6> <graph "bot" (id 4) >
penalty for edge <edge 6> is 0
	<edge 7> <graph "top" (id 1) >
	<edge 7> <graph "mid1" (id 2) >
penalty for edge <edge 7> is 0
0.448663175367848
